# Fridge Food Detection

Sarvesh Somasundaram, Taha Haveliwala, Arul Sharma

## Imports

In [ ]:
%tensorflow_version 1.x
!pip install q keras==2.1.6
import tensorflow as tf
import keras as K
print(tf.__version__)
print(K.__version__)

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount("/content/drive")

In [ ]:
from keras.callbacks import Callback
from keras.backend import clear_session
from keras.models import Model, load_model
from keras.layers import Dense, Input, Flatten
from keras.applications import ResNet50
import pandas as pd

import os
import cv2
import numpy as np
import pandas as pd

from collections import Counter

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

## Model Training

In [ ]:
def build_model(mode, model_path = None):

    clear_session()

    if mode == 'train':
        img = Input(shape = (224, 224, 3)) #Setting the input shape of the image

        # Importing the model and removing the top layer
        model = ResNet50(include_top=False, 
                          weights='imagenet', 
                          input_tensor=img, 
                          input_shape=None, 
                          pooling='avg')

        final_layer = model.layers[-1].output

        dense_layer_1 = Dense(128, activation = 'relu')(final_layer) # Adding a final dense layer
        output_layer = Dense(8, activation = 'sigmoid')(dense_layer_1) # Adding an output layer with sigmoid activation and 8 outputs
        model = Model(input = img, output = output_layer)
        
        model.trainable = True

        set_trainable = False
        
        # Setting the specified layers as trainable
        for layer in model.layers:
            if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
                set_trainable = True
            if set_trainable:
                layer.trainable = True
            else:
                layer.trainable = False
        layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
        print(pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']))
          
    # This mode is for predicting with the model
    elif mode == 'inference':
        model = load_model(model_path)
    return model


In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy' , metrics = ['accuracy']) # Compiling the model

In [ ]:
def F1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='samples') # Computes the f1 score which is a metric that measures recall

class ComputeF1(Callback):
    
    def __init__(self):
        self.best_f1 = -1
        
    def on_epoch_end(self, epoch, logs={}):
        val_pred = np.round(self.model.predict(self.validation_data[0]))
        val_f1 = f1_score(self.validation_data[1], val_pred, average='samples') # Calculating the F1 Score at the end of each epoch to see if the model has improved enough to save
        print('Validation Average F1 Score: ', val_f1)
        
        if val_f1 > self.best_f1:
            print('Better F1 Score, Saving model...')
            self.model.save('model.h5') # Saving the model
            self.best_f1 = val_f1

# Processing the images and data from csv file
def load_data(df):
    
    trainX, testX, valX = [], [], []
    trainY, testY, valY = [], [], []
    
    for i in range(len(df)):
        
        item = df.loc[i][0]
        current_label = np.array((df.loc[i])[1:])
        
        path = os.path.join('/content/drive/My Drive/project/data/augimages2', item)
        list_of_imgs = [os.path.join(path, file) for file in os.listdir(path)]
        # Splitting the images into train, validation, and test sets
        train_set = list_of_imgs[:150]
        val_set = list_of_imgs[150:200]
        test_set = list_of_imgs[200:]
        
        for file in train_set:
            img = cv2.resize(cv2.cvtColor(cv2.imread(file, 1), cv2.COLOR_BGR2RGB), (224, 224))
            trainX.append(img)
            trainY.append(current_label)
        
        for file in val_set:
            img = cv2.resize(cv2.cvtColor(cv2.imread(file, 1), cv2.COLOR_BGR2RGB), (224, 224))
            valX.append(img)
            valY.append(current_label)
        
        for file in test_set:
            img = cv2.resize(cv2.cvtColor(cv2.imread(file, 1), cv2.COLOR_BGR2RGB), (224, 224))
            testX.append(img)
            testY.append(current_label)
            
    return (np.array(trainX), np.array(trainY), np.array(testX), 
            np.array(testY), np.array(valX), np.array(valY))


In [ ]:
if __name__ == '__main__':

    print('Loading Data...') # Running the load_data function
    df = pd.read_csv('/content/drive/My Drive/project/data/annotationsnew.csv') # importing the csv file
    print(df)
    trainX, trainY, testX, testY, valX, valY = load_data(df)
    print('Data Loaded.')

    ## Normalization
    trainX = trainX.astype(np.float32)
    testX = testX.astype(np.float32)
    valX = valX.astype(np.float32)

    trainY = trainY.astype(np.float32)
    testY = testY.astype(np.float32)
    valY = valY.astype(np.float32)

    # Computing the first F1 score
    f1_score_callback = ComputeF1()
    model = build_model('train') # Initializing the model

    # Training model
    model.fit(trainX, trainY, batch_size = 32, epochs = 15, validation_data = (valX, valY), callbacks = [f1_score_callback])

    # Compute test F1 Score
    model = load_model('model.h5')

    score = F1_score(testY, model.predict(testX).round())
    print('F1 Score =', score)

## Prediction

In [ ]:
# We used this to filter out the log outputs to make it wasier to read
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# These are the categories that the model was trained on
categories = ['apple', 'bread', 'brocolli', 'carrot', 'chicken', 'milk', 'orange', 'pork']

In [ ]:
image = Image.open('/content/download.jpg') # opening image
new_image = image.resize((224, 224)) # Resizing image to fit our model
new_image.save('test.jpg') # Saving new image for testing

In [ ]:
model = build_model('inference', model_path = '/content/model.h5') # Initialzing model

img = np.expand_dims(cv2.imread('/content/test.jpg', 1), axis = 0) # Preparing input image

prediction = np.round(model.predict(img)[0]) # Running the prediction
print(prediction)
labels = [categories[idx] for idx, current_prediction in enumerate(prediction) if current_prediction == 1] # Adding the labels
print('Prediction:', labels)